<a href="https://colab.research.google.com/github/quirogaez/capstone/blob/main/notebooks/03_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **03_EDA – Procesamiento Final de Imágenes: Balanceo de Clases**

## Importación y cargue de imágenes

Importar librerías

In [6]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Llamar los archivos

In [3]:
# Cargar imágenes
img_train = '/content/drive/MyDrive/Capstone/data/multiclass/train'
# img_vali = '/content/drive/MyDrive/Capstone/data/multiclass/val'
# img_test = '/content/drive/MyDrive/Capstone/data/multiclass/test'

# Ver archivos en la carpeta
df_train = os.listdir(img_train)
# df_vali = os.listdir(img_vali)
# df_test = os.listdir(img_test)

Cargar datasets

In [11]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    img_train, # Entrenamiento
    batch_size = 32,
    label_mode = 'categorical'
)
class_names = train_ds.class_names

# test_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     img_test, # Testeo
#     batch_size = 32,
#     label_mode = 'categorical'
# )

# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     img_vali, # Validación
#     batch_size = 32,
#     label_mode = 'categorical'
# )

Found 5143 files belonging to 3 classes.


Contar la cantidad de imágenes por clase

In [12]:
label_list = []

for _, labels in train_ds.unbatch():
    label_list.append(tf.argmax(labels).numpy())

counter = Counter(label_list)

for idx, class_name in enumerate(class_names):
    print(f'{class_name}: {counter[idx]} imágenes')

NORMAL: 1269 imágenes
PNEUMONIA-BACTERIAL: 2110 imágenes
PNEUMONIA-VIRAL: 1764 imágenes
